### Memory mit Flaggen auf Canvas

In [1]:
# Normalerweise werden import statements nur einmal ausgefuehrt
# Nachfolgende Cell_Magic fuehrt die import statements nochmals aus, 
# falls das zu importierende File modifiziert wurde
%load_ext autoreload
%autoreload 2

In [2]:
import memory as M

In [3]:
M.init(2*list('ABCD'))

new_game {'nplayers': 2, 'player_to_move': 0, 'matches': {0: [], 1: []}, 'layout': {0: 'A', 1: 'D', 2: 'C', 3: 'B', 4: 'A', 5: 'C', 6: 'D', 7: 'B'}, 'face_up': ()}


In [4]:
# statt print soll M.callback folgende Funktion sein:
def show_info(event, data):
    print(event)
    print('Matches:', M.state['matches'])   
    print('Cards face-up', M._cards_face_up())
    print('Player to move:', M.state['player_to_move'])   
    print('Verbleibende Karten:', M._remaining_cards())
    
M.set_callback(show_info)    

In [5]:
layout = M.state['layout']
layout

{0: 'A', 1: 'D', 2: 'C', 3: 'B', 4: 'A', 5: 'C', 6: 'D', 7: 'B'}

In [6]:
matches = [(i,j) for i,x in layout.items() for j,y in layout.items() if i < j and x == y]
matches

[(0, 4), (1, 6), (2, 5), (3, 7)]

In [7]:
picks1 = matches[:2] + [(matches[2][0], matches[3][1])]
picks2 = matches[2:] 
picks1, picks2

([(0, 4), (1, 6), (2, 7)], [(2, 5), (3, 7)])

In [8]:
for pick in picks1:
    M.face_up(*pick)
    M.face_down()

phase_1
Matches: {0: [], 1: []}
Cards face-up [(0, 'A'), (4, 'A')]
Player to move: 0
Verbleibende Karten: [0, 1, 2, 3, 4, 5, 6, 7]
phase_2
Matches: {0: ['A'], 1: []}
Cards face-up []
Player to move: 0
Verbleibende Karten: [1, 2, 3, 5, 6, 7]
phase_1
Matches: {0: ['A'], 1: []}
Cards face-up [(1, 'D'), (6, 'D')]
Player to move: 0
Verbleibende Karten: [1, 2, 3, 5, 6, 7]
phase_2
Matches: {0: ['A', 'D'], 1: []}
Cards face-up []
Player to move: 0
Verbleibende Karten: [2, 3, 5, 7]
phase_1
Matches: {0: ['A', 'D'], 1: []}
Cards face-up [(2, 'C'), (7, 'B')]
Player to move: 0
Verbleibende Karten: [2, 3, 5, 7]
phase_2
Matches: {0: ['A', 'D'], 1: []}
Cards face-up []
Player to move: 1
Verbleibende Karten: [2, 3, 5, 7]


In [9]:
for pick in picks2:
    M.face_up(*pick)
    M.face_down()

phase_1
Matches: {0: ['A', 'D'], 1: []}
Cards face-up [(2, 'C'), (5, 'C')]
Player to move: 1
Verbleibende Karten: [2, 3, 5, 7]
phase_2
Matches: {0: ['A', 'D'], 1: ['C']}
Cards face-up []
Player to move: 1
Verbleibende Karten: [3, 7]
phase_1
Matches: {0: ['A', 'D'], 1: ['C']}
Cards face-up [(3, 'B'), (7, 'B')]
Player to move: 1
Verbleibende Karten: [3, 7]
phase_2
Matches: {0: ['A', 'D'], 1: ['C', 'B']}
Cards face-up []
Player to move: None
Verbleibende Karten: []


***
Wir schreiben nun eine Funktion `M.callback`, so dass der Spielzustand in einem Canvas-Widget dargestellt wird.
***

In [10]:
from ipycanvas import MultiCanvas
from draw_flags import draw_CH_flag, draw_BRD_flag, draw_BE_flag

ModuleNotFoundError: No module named 'draw_flags'

***
Unser Deck besteht nun nicht mehr aus Buchstaben, sondern aus Funktionen.
***

In [96]:
# vgl. deck = list('ABC') * 2
flags = [draw_CH_flag, draw_BRD_flag, draw_BE_flag] * 2
# flags[0]
flags[0].__name__

'draw_CH_flag'

In [75]:
FLAG_SIZE = 100
width, height = (3 * FLAG_SIZE, 2 * FLAG_SIZE)
mcanvas = MultiCanvas(2, width = width, height = height, 
                layout = {'border' : '2px solid black'}
               )

bg, fg = mcanvas
fg.line_width = 2
fg.stroke_style = 'black'
bg.fill_style = 'blue'

# horizonale Linie
fg.stroke_lines([(0, height/2), (width, height/2)])
# vertikale Linien
for i in range(1, 3):
    fg.stroke_lines([(i*width/3, 0), (i*width/3, height)])

display(mcanvas)

MultiCanvas(height=200, layout=Layout(border='2px solid black'), width=300)

In [76]:
# Teste Funktionen, zeichne Flaggen auf bg
for i, flag in enumerate(flags):
    x = (i % 3)  * FLAG_SIZE
    y = (i // 3) * FLAG_SIZE
    flag(bg, (x, y), FLAG_SIZE)

In [77]:
bg.clear()

In [109]:
def handle_event(canvas, event, data = None):
    
    def get_position(idx):
        return (FLAG_SIZE * (idx % 3), FLAG_SIZE * (idx // 3))
    
    canvas.fill_style = 'blue'
    
    if event == 'new_game':
        canvas.clear()
        for pos in [get_position(i) for i in range(6)]:
            canvas.fill_rect(*pos, FLAG_SIZE)
            
    if event == 'phase_1':
        (idx1, flag1), (idx2, flag2) = data
        pos1 = get_position(idx1)
        pos2 = get_position(idx2) 
        flag1(canvas, pos1, FLAG_SIZE)
        flag2(canvas, pos2, FLAG_SIZE)
    
    if event == 'phase_2':
        positions = [get_position(idx) for idx in data[1]]
        for pos in positions:
            canvas.clear_rect(*pos, FLAG_SIZE)
            
        if not data[0]:
            for pos in positions:
                canvas.fill_rect(*pos, FLAG_SIZE)

In [104]:
handle_event(, 'new_game')

In [105]:
picks = ((1, flags[1]), (3, flags[3]))
handle_event(mcanvas, 'phase_1', picks)

In [106]:
handle_event(mcanvas, 'phase_2', (True, (1,3),))

In [107]:
handle_event(mcanvas, 'phase_2', (False, (1,3),))

***
**Play Memory**
***

In [90]:
bg.clear()
display(mcanvas)

MultiCanvas(height=200, layout=Layout(border='2px solid black'), width=300)

In [111]:
callback = lambda event, data: handle_event(bg, event, data)
M.set_callback(callback)
M.init(flags)

In [112]:
layout = M.state['layout']
matches = [(i,j) for i,x in layout.items() for j,y in layout.items() if i < j and x == y]
matches

[(0, 2), (1, 3), (4, 5)]

In [113]:
M.face_up(4,5)

AttributeError: module 'memory' has no attribute 'face_up'

In [ ]:
M.face_down()

In [93]:
{player:[m.__name__ for m in matches] for player, matches in M.state['matches'].items()}

{0: [], 1: []}